<a href="https://colab.research.google.com/github/YasserJxxxx/RNN_Traductor/blob/main/traductor3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import os
import re
import unicodedata

# --- 1. Configuración ---
# El nombre exacto de tu archivo subido
FILE_IN = "Parejas de oraciones en EspañolInglés - 2025-11-18.tsv"

# ¡CAMBIO! El archivo de salida ahora es de 1 millón
FILE_OUT = "tatoeba_limpio_1M.csv"
# ¡CAMBIO! Aumentamos el número de muestras
N_SAMPLES = 1000000

# --- 2. Función de limpieza (sin tildes) ---
def limpiar_texto_sin_tildes(texto):
    texto = str(texto).lower()
    # Quitar tildes
    texto_normalizado = unicodedata.normalize('NFD', texto)
    texto_sin_tildes = texto_normalizado.encode('ascii', 'ignore').decode('utf-8')
    # Limpiar puntuación
    texto_sin_tildes = re.sub(r"([?.!,¿])", r" \1 ", texto_sin_tildes)
    texto_sin_tildes = re.sub(r"[^a-zA-Z0-9?.!]+", " ", texto_sin_tildes)
    texto_sin_tildes = texto_sin_tildes.strip()
    return texto_sin_tildes

# --- 3. Proceso de Lectura y Limpieza ---
print(f"Iniciando el procesamiento de '{FILE_IN}'...")

try:
    # --- Cargar el archivo .tsv ---
    df = pd.read_csv(
        FILE_IN,
        sep='\t',
        header=None,
        names=['espanol', 'ingles'],
        usecols=[0, 1],
        on_bad_lines='skip'
    )

    print(f"Se encontraron {len(df)} pares de frases en total.")
    df = df.dropna()

    # --- 4. Muestreo y Limpieza ---
    if len(df) > N_SAMPLES:
        print(f"Tomando una muestra aleatoria de {N_SAMPLES} pares...")
        df_sample = df.sample(n=N_SAMPLES, random_state=42)
    else:
        print(f"Se usarán todos los {len(df)} pares disponibles.")
        df_sample = df

    print("Limpiando textos (quitando tildes)...")
    df_sample['espanol'] = df_sample['espanol'].apply(limpiar_texto_sin_tildes)
    df_sample['ingles'] = df_sample['ingles'].apply(limpiar_texto_sin_tildes)

    df_sample = df_sample.dropna()
    df_sample = df_sample[
        (df_sample['espanol'].str.len() > 0) &
        (df_sample['ingles'].str.len() > 0)
    ]

    # --- 5. Guardar CSV Local ---
    df_sample.to_csv(FILE_OUT, index=False)

    print(f"\n¡Éxito! Dataset limpio guardado localmente en: {FILE_OUT}")
    print(f"Total de pares procesados: {len(df_sample)}")
    print("\nVista previa de los datos:")
    print(df_sample.head())

except FileNotFoundError:
    print(f"¡ERROR DE ARCHIVO NO ENCONTRADO!")
    print(f"Asegúrate de haber subido el archivo '{FILE_IN}' a Colab.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Iniciando el procesamiento de 'Parejas de oraciones en EspañolInglés - 2025-11-18.tsv'...
Se encontraron 275858 pares de frases en total.
Se usarán todos los 275858 pares disponibles.
Limpiando textos (quitando tildes)...

¡Éxito! Dataset limpio guardado localmente en: tatoeba_limpio_1M.csv
Total de pares procesados: 275858

Vista previa de los datos:
  espanol                     ingles
0    2481          intentemos algo !
1    2482  tengo que irme a dormir .
2    2483       que estas haciendo ?
3    2483       que estas haciendo ?
4    2483       que estas haciendo ?


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import re
import pickle
import csv
from google.colab import drive
import unicodedata

# --- 0. Montar Google Drive ---
drive.mount('/content/drive')

# --- 1. Configuración de Rutas ---
# Lee el archivo de 1M que creó la Celda 1
DATASET_FILE = "tatoeba_limpio_1M.csv"

# RUTA DE SALIDA (Guardado)
RUTA_BASE_DRIVE = "/content/drive/My Drive/TraductorRNN_Tatoeba_1M/"

if not os.path.exists(RUTA_BASE_DRIVE):
    os.makedirs(RUTA_BASE_DRIVE)
    print(f"Carpeta creada en: {RUTA_BASE_DRIVE}")

# Archivos que se guardarán/cargarán de Drive
MODEL_FILE = os.path.join(RUTA_BASE_DRIVE, 'traductor_rnn.keras')
TOKENIZER_SPA_FILE = os.path.join(RUTA_BASE_DRIVE, 'tokenizer_spa.pkl')
TOKENIZER_ENG_FILE = os.path.join(RUTA_BASE_DRIVE, 'tokenizer_eng.pkl')
MODEL_PARAMS_FILE = os.path.join(RUTA_BASE_DRIVE, 'model_params.pkl')
FEEDBACK_FILE = os.path.join(RUTA_BASE_DRIVE, 'feedback.csv')

# --- ¡¡CAMBIOS PARA AHORRAR MEMORIA!! ---
EMBEDDING_DIM = 256
LATENT_DIM = 256
# 1. Reducimos el tamaño del lote
BATCH_SIZE = 32
EPOCHS = 5
# 2. Añadimos una longitud MÁXIMA para truncar frases largas
MAX_SEQUENCE_LEN = 50
# --- FIN DE CAMBIOS DE MEMORIA ---

# --- 2. Funciones de Preprocesamiento ---
def limpiar_texto_sin_tildes(texto):
    """Función de limpieza para la ENTRADA DEL USUARIO y FEEDBACK."""
    texto = str(texto).lower()
    texto_normalizado = unicodedata.normalize('NFD', texto)
    texto_sin_tildes = texto_normalizado.encode('ascii', 'ignore').decode('utf-8')
    texto_sin_tildes = re.sub(r"([?.!,¿])", r" \1 ", texto_sin_tildes)
    texto_sin_tildes = re.sub(r"[^a-zA-Z0-9?.!]+", " ", texto_sin_tildes)
    texto_sin_tildes = texto_sin_tildes.strip()
    return texto_sin_tildes

def preparar_datos(dataset_path):
    """Carga el dataset LOCAL y combina el feedback de DRIVE."""
    print(f"Cargando y preparando datos desde {dataset_path}...")
    try:
        df_main = pd.read_csv(dataset_path)
    except FileNotFoundError:
        print(f"¡ERROR! No se encontró el dataset en: {dataset_path}")
        print("Por favor, asegúrate de ejecutar la 'Celda 1: Procesar Archivos' primero.")
        return None, None, None, None

    # Combinar con feedback
    if os.path.exists(FEEDBACK_FILE) and os.path.getsize(FEEDBACK_FILE) > 50:
        print(f"¡Feedback encontrado en Drive! Combinando...")
        try:
            df_feedback = pd.read_csv(FEEDBACK_FILE)
            df_feedback['espanol'] = df_feedback['espanol'].apply(limpiar_texto_sin_tildes)
            df_feedback['ingles'] = df_feedback['ingles'].apply(limpiar_texto_sin_tildes)
            df = pd.concat([df_main, df_feedback], ignore_index=True)
            df = df.drop_duplicates(subset=['espanol'])
        except Exception as e:
            print(f"Error al combinar feedback: {e}")
            df = df_main
    else:
        print("No se encontró feedback. Usando solo el dataset original.")
        df = df_main

    # Continuar con el preprocesamiento...
    espanol_textos = [str(txt) for txt in df['espanol']]
    ingles_textos = [f"[start] {str(txt)} [end]" for txt in df['ingles']]

    tokenizer_spa = Tokenizer(filters='', oov_token='[UNK]')
    tokenizer_spa.fit_on_texts(espanol_textos)

    tokenizer_eng = Tokenizer(filters='', oov_token='[UNK]')
    tokenizer_eng.fit_on_texts(ingles_textos)

    with open(TOKENIZER_SPA_FILE, 'wb') as f:
        pickle.dump(tokenizer_spa, f)
    with open(TOKENIZER_ENG_FILE, 'wb') as f:
        pickle.dump(tokenizer_eng, f)

    print(f"Tokenizers guardados en {RUTA_BASE_DRIVE}")
    return espanol_textos, ingles_textos, tokenizer_spa, tokenizer_eng

# --- 3. Función de Entrenamiento ---
def entrenar_y_guardar_modelo(espanol_textos, ingles_textos, tokenizer_spa, tokenizer_eng):
    print("Iniciando construcción y entrenamiento del modelo...")

    encoder_input_seq = tokenizer_spa.texts_to_sequences(espanol_textos)
    decoder_input_seq = tokenizer_eng.texts_to_sequences(ingles_textos)

    decoder_target_seq = [seq[1:] for seq in decoder_input_seq]

    # --- ¡CAMBIO! Usamos la longitud fija ---
    max_len_spa = MAX_SEQUENCE_LEN
    max_len_eng = MAX_SEQUENCE_LEN

    with open(MODEL_PARAMS_FILE, 'wb') as f:
        pickle.dump({'max_len_spa': max_len_spa, 'max_len_eng': max_len_eng}, f)

    # --- ¡CAMBIO! Añadimos 'truncating' para cortar frases largas ---
    encoder_input_data = pad_sequences(encoder_input_seq, maxlen=max_len_spa, padding='post', truncating='post')
    decoder_input_data = pad_sequences(decoder_input_seq, maxlen=max_len_eng, padding='post', truncating='post')
    decoder_target_data = pad_sequences(decoder_target_seq, maxlen=max_len_eng, padding='post', truncating='post')

    # Corregimos el 'shape' para sparse_categorical
    decoder_target_data = np.expand_dims(decoder_target_data, -1)

    num_decoder_tokens = len(tokenizer_eng.word_index) + 1
    # --- FIN DE CAMBIOS ---

    print(f"Vocabulario Español: {len(tokenizer_spa.word_index) + 1} tokens")
    print(f"Vocabulario Inglés: {num_decoder_tokens} tokens")
    print(f"Max. secuencia (FORZADA): {MAX_SEQUENCE_LEN}")

    num_encoder_tokens = len(tokenizer_spa.word_index) + 1

    # --- Arquitectura del Modelo ---
    encoder_inputs = Input(shape=(None,), name='encoder_input')
    enc_embedding_layer = Embedding(num_encoder_tokens, EMBEDDING_DIM, name='encoder_embedding')
    enc_embedding_tensor = enc_embedding_layer(encoder_inputs)
    encoder_lstm_layer = LSTM(LATENT_DIM, return_state=True, name='encoder_lstm')
    _, state_h, state_c = encoder_lstm_layer(enc_embedding_tensor)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None,), name='decoder_input')
    dec_embedding_layer = Embedding(num_decoder_tokens, EMBEDDING_DIM, name='decoder_embedding')
    dec_embedding_tensor = dec_embedding_layer(decoder_inputs)
    decoder_lstm_layer = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm_layer(dec_embedding_tensor, initial_state=encoder_states)
    decoder_dense_layer = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense_layer(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print("\nEntrenando... (esto tardará varias horas con 1M de filas)")
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=BATCH_SIZE, # Batch size reducido
        epochs=EPOCHS,
        validation_split=0.2
    )

    model.save(MODEL_FILE)
    print(f"Modelo guardado en {MODEL_FILE}")
    return model

# --- 4. Funciones de Inferencia ---
def crear_modelos_de_inferencia(modelo_entrenado):
    print("Creando modelos de inferencia...")

    encoder_model_input = Input(shape=(None,), name='encoder_input_inf')
    enc_embedding_layer_inf = modelo_entrenado.get_layer('encoder_embedding')
    encoder_lstm_layer_inf = modelo_entrenado.get_layer('encoder_lstm')
    enc_emb_tensor_inf = enc_embedding_layer_inf(encoder_model_input)
    _, state_h_inf, state_c_inf = encoder_lstm_layer_inf(enc_emb_tensor_inf)
    encoder_model = Model(encoder_model_input, [state_h_inf, state_c_inf])

    decoder_model_input = Input(shape=(1,), name='decoder_input_inf')
    decoder_state_h_input = Input(shape=(LATENT_DIM,), name='decoder_state_h_input')
    decoder_state_c_input = Input(shape=(LATENT_DIM,), name='decoder_state_c_input')
    decoder_states_inputs_inf = [decoder_state_h_input, decoder_state_c_input]

    dec_embedding_layer_inf = modelo_entrenado.get_layer('decoder_embedding')
    decoder_lstm_layer_inf = modelo_entrenado.get_layer('decoder_lstm')
    decoder_dense_layer_inf = modelo_entrenado.get_layer('decoder_dense')

    dec_emb_tensor_inf = dec_embedding_layer_inf(decoder_model_input)
    dec_outputs_inf, dec_state_h_inf, dec_state_c_inf = decoder_lstm_layer_inf(
        dec_emb_tensor_inf, initial_state=decoder_states_inputs_inf
    )
    dec_final_outputs = decoder_dense_layer_inf(dec_outputs_inf)

    decoder_model = Model(
        [decoder_input_inf] + decoder_states_inputs_inf,
        [dec_final_outputs, dec_state_h_inf, dec_state_c_inf]
    )

    return encoder_model, decoder_model

def traducir_frase(frase_es, encoder_model, decoder_model, tokenizer_spa, tokenizer_eng, max_len_spa, max_len_eng):
    frase_limpia = limpiar_texto_sin_tildes(frase_es)
    input_seq_raw = tokenizer_spa.texts_to_sequences([frase_limpia])

    if not np.any(input_seq_raw[0]) or (len(input_seq_raw[0]) > 0 and all(token == tokenizer_spa.word_index['[UNK]'] for token in input_seq_raw[0])):
         return "[Error: No reconozco ninguna de esas palabras]"

    # Usamos max_len_spa (que ahora es 50) para el padding
    input_seq = pad_sequences(input_seq_raw, maxlen=max_len_spa, padding='post', truncating='post')

    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_eng.word_index['[start]']
    stop_condition = False
    decoded_sentence = ""
    idx_a_palabra_eng = {v: k for k, v in tokenizer_eng.word_index.items()}

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = idx_a_palabra_eng.get(sampled_token_index, '[UNK]')

        # Usamos max_len_eng (que ahora es 50) para el límite de traducción
        if (sampled_word == '[end]' or
           sampled_word == '[UNK]' or
           len(decoded_sentence.split()) > max_len_eng):
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# --- 5. Bucle Principal de Ejecución ---
def main():
    if os.path.exists(MODEL_FILE) and os.path.exists(TOKENIZER_SPA_FILE):
        print("Cargando modelo y tokenizers existentes desde Drive...")
        modelo_entrenado = load_model(MODEL_FILE)

        with open(TOKENIZER_SPA_FILE, 'rb') as f:
            tokenizer_spa = pickle.load(f)
        with open(TOKENIZER_ENG_FILE, 'rb') as f:
            tokenizer_eng = pickle.load(f)
        with open(MODEL_PARAMS_FILE, 'rb') as f:
            params = pickle.load(f)
            max_len_spa = params['max_len_spa']
            max_len_eng = params['max_len_eng']

    else:
        print("No se encontró un modelo en Drive. Entrenando uno nuevo...")
        datos = preparar_datos(DATASET_FILE)
        if datos[0] is None:
            return
        spa_text, eng_text, tokenizer_spa, tokenizer_eng = datos
        modelo_entrenado = entrenar_y_guardar_modelo(
            spa_text, eng_text, tokenizer_spa, tokenizer_eng
        )
        with open(MODEL_PARAMS_FILE, 'rb') as f:
            params = pickle.load(f)
            max_len_spa = params['max_len_spa']
            max_len_eng = params['max_len_eng']

    encoder_model_inf, decoder_model_inf = crear_modelos_de_inferencia(
        modelo_entrenado,
    )
    print("¡Traductor listo!")

    print("\n--- Traductor Interactivo (RNN con 1M de datos) ---")
    print("Escribe 'salir' para terminar.")

    while True:
        frase_original = input("\nEscribe en español (sin tildes): ")
        if frase_original.lower() == 'salir':
            break
        if not frase_original.strip():
            continue

        traduccion = traducir_frase(
            frase_original,
            encoder_model_inf,
            decoder_model_inf,
            tokenizer_spa,
            tokenizer_eng,
            max_len_spa,
            max_len_eng
        )
        print(f"Traducción: {traduccion}")

        feedback = input("¿Traducción correcta? (s/n): ").lower() # Corregido .cuerpo() a .lower()

        if feedback == 'n':
            correccion = input(f"Escribe la traducción correcta para '{frase_original}': ")
            nueva_fila = [frase_original, correccion]
            file_exists = os.path.exists(FEEDBACK_FILE)

            with open(FEEDBACK_FILE, 'a', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(['espanol', 'ingles'])
                writer.writerow(nueva_fila)
            print(f"¡Gracias! Corrección guardada en '{FEEDBACK_FILE}'.")
        elif feedback == 's':
            print("¡Genial! 👍")

# --- ¡Ejecutar todo! ---
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No se encontró un modelo en Drive. Entrenando uno nuevo...
Cargando y preparando datos desde tatoeba_limpio_1M.csv...
No se encontró feedback. Usando solo el dataset original.
Tokenizers guardados en /content/drive/My Drive/TraductorRNN_Tatoeba_1M/
Iniciando construcción y entrenamiento del modelo...
Vocabulario Español: 254307 tokens
Vocabulario Inglés: 48853 tokens
Max. secuencia (FORZADA): 50

Entrenando... (esto tardará varias horas con 1M de filas)
Epoch 1/5
5756/6897 ━━━━━━━━━━━━━━━━━━━━ 3:53 205ms/step - accuracy: 0.8627 - loss: 1.1248